<a href="https://colab.research.google.com/github/csaswanth/ALE/blob/main/t5_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from transformers import T5ForConditionalGeneration,AutoTokenizer,TrainingArguments,Trainer

In [ ]:
from numpy.random.mtrand import RandomState
from torch.utils.data.dataset import random_split
dataset=load_dataset("csv",data_files="/content/drive/MyDrive/Datasets/Data-Pool1ww.csv",split="train")


In [ ]:
dataset=dataset.train_test_split(test_size=0.1, seed=42)
train_dataset=dataset['train']
test_dataset=dataset['test']
# from sklearn.model_selection import train_test_split

# train, test=train_test_split(dataset, test_size=0.1, random_state=2)

In [ ]:
train_dataset

Dataset({
    features: ['Original', 'Final'],
    num_rows: 129097
})

In [ ]:
test_dataset

Dataset({
    features: ['Original', 'Final'],
    num_rows: 14345
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('t5-base')



Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,


In [ ]:
def tokenize(batch):
    tokenized_input = tokenizer(batch['Original'], padding='max_length', truncation=True, max_length=180)
    tokenized_label = tokenizer(batch['Final'], padding='max_length', truncation=True, max_length=180)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=180)
val_dataset = test_dataset.map(tokenize, batched=True, batch_size=180)

train_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

  0%|          | 0/80 [00:00<?, ?ba/s]

In [ ]:
train_dataset

Dataset({
    features: ['Original', 'Final', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 129097
})

In [ ]:
from transformers.trainer_callback import EarlyStoppingCallback

model = T5ForConditionalGeneration.from_pretrained('t5-base')

output_dir = '/content/drive/MyDrive/model_saving'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.0001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=30000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    #run_name='run_name', # Wandb run name
    logging_steps=1000, # How often to log loss to wandb
    eval_steps=1000, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False, # Best model is the one with the lowest loss, not highest.
    #push_to_hub=True,
    #push_to_hub_model_id="finetune-t5-for-grammar-correction"
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "su

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Original, Final. If Original, Final are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 129097
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 48414
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed 

Step,Training Loss,Validation Loss
1000,0.100300,0.012123
2000,0.013500,0.010607
3000,0.012200,0.010094
4000,0.011400,0.010345
5000,0.011500,0.010009
6000,0.011100,0.009288
7000,0.010800,0.009432
8000,0.010400,0.009165
9000,0.010600,0.008562
10000,0.010300,0.008513


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Original, Final. If Original, Final are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 14345
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Original, Final. If Original, Final are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 14345
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Original, Final. If Original, Final are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num

TrainOutput(global_step=48414, training_loss=0.00924328304791818, metrics={'train_runtime': 14894.0608, 'train_samples_per_second': 26.003, 'train_steps_per_second': 3.251, 'total_flos': 8.29138747474944e+16, 'train_loss': 0.00924328304791818, 'epoch': 3.0})

In [ ]:
import datetime
import pytz

In [ ]:
#for dynamic variable
UTC=pytz.utc
IST=pytz.timezone('Asia/Kolkata')
dt=datetime.datetime.now(IST)
dt2=dt.strftime("%d-%m-%y")
dt3=dt.strftime("%H:%M:%S")

In [ ]:
trainer.save_model(output_dir + f'/model-{dt2}-{dt3}')

Saving model checkpoint to /content/drive/MyDrive/model_saving/model-21-10-22-21:22:08
Configuration saved in /content/drive/MyDrive/model_saving/model-21-10-22-21:22:08/config.json
Model weights saved in /content/drive/MyDrive/model_saving/model-21-10-22-21:22:08/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/model_saving/model-21-10-22-21:22:08/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/model_saving/model-21-10-22-21:22:08/special_tokens_map.json


In [ ]:
# !tar -vf "/content/drive/MyDrive/model_saving/output_file_name.tar.gz" "/content/drive/MyDrive/model_saving/model-15-09-22-14:10:07"     #[run this cell to create tar.gz files]